# ECサイトなどのカタログデータの処理・分析の小技集

一部のECサイトでは、販売商品のカタログデータをAPIにより取得することができます。このようなデータには、商品名、商品の補足説明、価格などの情報が含まれています。ここでは、このようなECサイトのカタログデータを処理・分析するためのpythonの小技をまとめました。ここで用いるデータは「楽天ブックス書籍検索API」を利用してダウンロードした書籍情報データです。

In [72]:
# japanize_matplotlibが未インストール状態の場合に、コメント解除して本セルを単独で実行する（先頭に!が必要）
#!pip install japanize-matplotlib

In [73]:
# Googleドライブをマウントする場合、２行のコメント解除して本セルを単独で実行する
#from google.colab import drive
#drive.mount('/content/drive')

In [74]:
# ライブラリのインポート
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font=['IPAexGothic'])
import japanize_matplotlib # matplotlib日本語
from google.colab import files # 画像ファイルダウンロード用

parquetファイルからデータフレームに読み込む

In [75]:
df01 = pd.read_parquet('/content/drive/My Drive/Colab Notebooks/rakuten_books.parquet')
df01.iloc[1000:1003,:]

,title,titleKana,subTitle,subTitleKana,seriesName,seriesNameKana,contents,author,authorKana,publisherName,size,isbn,itemCaption,salesDate,itemPrice,listPrice,discountRate,discountPrice,itemUrl,affiliateUrl,smallImageUrl,mediumImageUrl,largeImageUrl,chirayomiUrl,availability,postageFlag,limitedFlag,reviewCount,booksGenreId,reviewAverage
1000,魔法科高校の劣等生 古都内乱編2,マホウカコウコウノレットウセイ コトナイランヘン2,,,電撃コミックスNEXT,デンゲキコミックスネクスト,,佐島 勤/柚木N’,サトウ ツトム/ユズキエヌダッシュ,KADOKAWA,コミック,9784049132366,,2020年06月10日,715,0,0,0,https://books.rakuten.co.jp/rb/16310599/,,https://thumbnail.image.rakuten.co.jp/@0_mall/...,https://thumbnail.image.rakuten.co.jp/@0_mall/...,https://thumbnail.image.rakuten.co.jp/@0_mall/...,,1,0,0,1,001001012,5.0
1001,文豪春秋,ブンゴウシュンジュウ,,,,,,ドリヤス工場,ドリヤスコウジョウ,文藝春秋,単行本,9784163912141,漫画版文壇事件簿。,2020年06月12日,935,0,0,0,https://books.rakuten.co.jp/rb/16316558/,,https://thumbnail.image.rakuten.co.jp/@0_mall/...,https://thumbnail.image.rakuten.co.jp/@0_mall/...,https://thumbnail.image.rakuten.co.jp/@0_mall/...,,1,0,0,0,001008022001,0.0
1002,SNOOPY マルチに使えるBIGピクニックバッグ BOOK,スヌーピーマルチニツカエルビッグピクニックバッグブック,,,,,,,,宝島社,ムックその他,9784299003232,,2020年05月02日,2310,0,0,0,https://books.rakuten.co.jp/rb/16249203/,,https://thumbnail.image.rakuten.co.jp/@0_mall/...,https://thumbnail.image.rakuten.co.jp/@0_mall/...,https://thumbnail.image.rakuten.co.jp/@0_mall/...,,1,1,0,10,001010014006,4.1


In [76]:
# データフレームの情報を表示する
df01.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 30 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3000 non-null   object 
 1   titleKana       3000 non-null   object 
 2   subTitle        3000 non-null   object 
 3   subTitleKana    3000 non-null   object 
 4   seriesName      3000 non-null   object 
 5   seriesNameKana  3000 non-null   object 
 6   contents        3000 non-null   object 
 7   author          3000 non-null   object 
 8   authorKana      3000 non-null   object 
 9   publisherName   3000 non-null   object 
 10  size            3000 non-null   object 
 11  isbn            3000 non-null   object 
 12  itemCaption     3000 non-null   object 
 13  salesDate       3000 non-null   object 
 14  itemPrice       3000 non-null   int64  
 15  listPrice       3000 non-null   int64  
 16  discountRate    3000 non-null   int64  
 17  discountPrice   3000 non-null   i

In [77]:
# 列を絞り込む
focus_cols=['title','subTitle','seriesName','contents','author','publisherName','size','itemCaption','salesDate','itemPrice','reviewCount','reviewAverage']
df1 = df01.loc[:,focus_cols]

**テキスト項目の文字列検索**

まず、テキスト情報からあるキーワードを含む行を抽出するケースを考えます。pandasのSeries.str.contains()を用います。これにより、指定した文字列を含む行のみを取り出すことができます。

In [78]:
# テキスト項目で特定の文字列を検索する
col_name = 'title'
ch_kw = '資産'
df201 = df1[df1[col_name].str.contains(ch_kw)]
print('該当するデータ件数 = ',len(df201))
df201.head(2)

該当するデータ件数 =  3


,title,subTitle,seriesName,contents,author,publisherName,size,itemCaption,salesDate,itemPrice,reviewCount,reviewAverage
582,2020-2021年版 みんなが欲しかった！ FPの教科書1級 Vol．1 ライフプランニン...,,,,滝澤ななみ監修・TAC株式会社（FP講座）著,TAC出版,単行本,’２０年９月、’２１年１月・５月試験対応。,2020年06月03日,4180,0,0.00
2444,山崎元の“やってはいけない”資産運用,もう銀行・証券会社にだまされない！,TJ MOOK,,山崎元,宝島社,ムックその他,,2019年01月25日,1100,3,4.33


次に、２つのキーワードについて、その両方を含む行を取り出したい（and条件）場合は、一つのキーワードで絞りこんだデータフレームに対してもう一つのキーワードでさらに絞りこみをかけることでできます。

In [79]:
# テキスト項目で特定の複数の文字列をand検索する
col_name = 'itemCaption'
ch_kw1 = '成功'; ch_kw2 = '金'
df202 = df1[df1[col_name].str.contains(ch_kw1)]
df203 = df202[df202[col_name].str.contains(ch_kw2)]
print('該当するデータ件数 = ',len(df203))
df203.head(2)

該当するデータ件数 =  1


,title,subTitle,seriesName,contents,author,publisherName,size,itemCaption,salesDate,itemPrice,reviewCount,reviewAverage
1746,進化する里山資本主義,,,,藻谷浩介,ジャパンタイムズ出版,単行本,金銭的利益最優先の「マネー資本主義」のアンチテーゼとして、「里山資本主義」が提唱されてから７...,2020年05月01日,1980,1,5.0


一方、２つのキーワードのどちらかが含まれる行を抽出したい（or条件）場合は、元のデータフレームを個別のキーワードで絞りこんだ結果を縦結合することでできます。これは検索対象の列が異なる場合にも応用できます。どちらかの検索結果がゼロ行でも特に問題はありません。

In [80]:
# テキスト項目で特定の複数の文字列をor検索する
col_name = 'title'
ch_kw1 = '資産'; ch_kw2 = '財産'
df204 = df1[df1[col_name].str.contains(ch_kw1)]
df205 = df1[df1[col_name].str.contains(ch_kw2)]
df206 = pd.concat([df204,df205])
print('該当するデータ件数 = ',len(df206))
df206.head(2)

該当するデータ件数 =  6


,title,subTitle,seriesName,contents,author,publisherName,size,itemCaption,salesDate,itemPrice,reviewCount,reviewAverage
582,2020-2021年版 みんなが欲しかった！ FPの教科書1級 Vol．1 ライフプランニン...,,,,滝澤ななみ監修・TAC株式会社（FP講座）著,TAC出版,単行本,’２０年９月、’２１年１月・５月試験対応。,2020年06月03日,4180,0,0.00
2444,山崎元の“やってはいけない”資産運用,もう銀行・証券会社にだまされない！,TJ MOOK,,山崎元,宝島社,ムックその他,,2019年01月25日,1100,3,4.33


**特定の数値項目で条件検索する**

今度は数値データが入った列について、ある条件を満たす行を取り出すことを考えます。例えば値がゼロより大きい行を抽出するには、df[df['列名']>0]とする書き方と、df.query('列名>0')という書き方があります。

In [81]:
# 数値変数で条件検索する
ch_col='reviewCount'; s_val=500
df31 = df1[df1[ch_col] >= s_val]
#別の記述法
#df31 = df1.query('{0} >= @s_val'.format(ch_col))
print('該当するケース数: ',len(df31))
df31.head(3)

該当するケース数:  3


,title,subTitle,seriesName,contents,author,publisherName,size,itemCaption,salesDate,itemPrice,reviewCount,reviewAverage
424,FACTFULNESS（ファクトフルネス）,10の思い込みを乗り越え、データを基に世界を正しく見る習慣,,,ハンス・ロスリング/オーラ・ロスリング,日経BP,単行本,ここ数十年間、わたしは何千もの人々に、貧困、人口、教育、エネルギーなど世界にまつわる数多くの...,2019年01月12日,1980,752,4.40
458,ぼくはイエローでホワイトで、ちょっとブルー,,,,ブレイディ みかこ,新潮社,単行本,大人の凝り固まった常識を、子どもたちは軽く飛び越えていく。世界の縮図のような「元・底辺中学校...,2019年06月21日,1485,549,4.49
1653,メモの魔力,The Magic of Memo,NewsPicks Book,,前田裕二,幻冬舎,単行本,僕にとってメモとは、生き方そのものです。メモによって世界を知り、アイデアが生まれる。メモによ...,2018年12月24日,1540,550,4.14


In [82]:
#数値検索の別の記述法
ch_col='reviewAverage'; s_val=5
df32 = df1.query('{0} >= @s_val'.format(ch_col))
print('該当するケース数: ',len(df32))
df32.head(2)

該当するケース数:  360


,title,subTitle,seriesName,contents,author,publisherName,size,itemCaption,salesDate,itemPrice,reviewCount,reviewAverage
3,YUZU’LL BE BACK 2,羽生結弦写真集2019〜2020,,,,スポーツニッポン新聞社,単行本,,2020年06月,2750,4,5.0
12,cookpadLive公式レシピ 和牛キッチン 川西シェフ・助手水田,,,,和牛/CookpadTV株式会社,ヨシモトブックス,単行本,,2020年06月19日,1320,6,5.0


**特定の列項目が空でないケースを抽出する**

次に、テキストからなる特定の列項目が空でないケースを抽出する方法を考えます。テキスト情報で何も書かれていないのは空文字列('')なので、まず空文字列を欠損値(NaN)に置き換えます。その後で欠損値を含む行を削除します。

In [83]:
# 列名を指定する
col_name = 'contents'
# データフレームをコピーする
df211 = df1.copy()
#空白をまずNaNに置き換え
df211[col_name].replace('', np.nan, inplace=True)
#Nanを削除 inplace=Trueでdfが上書きされる。
df211.dropna(subset=[col_name], inplace=True)
print('該当するデータ件数 = ',len(df211))
df211.head(2)

該当するデータ件数 =  36


,title,subTitle,seriesName,contents,author,publisherName,size,itemCaption,salesDate,itemPrice,reviewCount,reviewAverage
39,BARFOUT！（vol．298（JULY 20）,Culture Magazine From Shi,Brown’s books,高橋海人（king＆Prince）高橋真宇、カイ＆リョウガ＆,,ブラウンズブックス,単行本,,2020年06月,968,3,5.0
126,COTTON FRIEND SEWING（vol．4）,,レディブティックシリーズ,元気になれる、夏の服,,ブティック社,ムックその他,,2020年06月15日,1430,0,0.0


**定型テキストの出現頻度を求める**

テキスト情報の列の中でも、出現する文字列のパターンが限定されている場合があります。いわゆる「カテゴリカルデータ」に相当するものです。その出現頻度を知りたいことはよくあります。これにはcollectionsライブラリのCounter()メソッドを用います。

In [84]:
# 列名を指定する
import pprint
col_name = 'publisherName'
import collections
c_count = collections.Counter(df1[col_name])
pprint.pprint(c_count, compact=True)

Counter({'講談社': 265,
         'KADOKAWA': 246,
         '集英社': 187,
         '小学館': 161,
         '宝島社': 81,
         'スクウェア・エニックス': 69,
         '白泉社': 46,
         '秋田書店': 44,
         '幻冬舎': 37,
         '朝日新聞出版': 36,
         '双葉社': 36,
         '新潮社': 36,
         'ダイヤモンド社': 35,
         'アルファポリス': 35,
         'TAC出版': 32,
         '文藝春秋': 31,
         'ワニブックス': 29,
         '扶桑社': 29,
         '一迅社': 29,
         '光文社': 29,
         '日経BP': 28,
         '学研プラス': 25,
         '旺文社': 25,
         'SBクリエイティブ': 23,
         '中央公論新社': 21,
         '主婦の友社': 20,
         '祥伝社': 19,
         '徳間書店': 18,
         'メディック\u3000メディア': 17,
         '技術評論社': 17,
         'NHK出版': 17,
         'サンマーク出版': 16,
         'マガジンハウス': 15,
         '医学書院': 15,
         '中央法規出版': 15,
         '芳文社': 15,
         '中央経済社': 15,
         'コアミックス': 14,
         'ぴあ': 14,
         '河出書房新社': 14,
         '東洋経済新報社': 13,
         '岩波書店': 13,
         '早川書房': 12,
         '医学通信社': 12,
         '日経BP 日本経済新聞出版本部':

In [85]:
# カウント上位のリストを得る
p_names, p_counts = zip(*c_count.most_common(10))
print(p_names)

('講談社', 'KADOKAWA', '集英社', '小学館', '宝島社', 'スクウェア・エニックス', '白泉社', '秋田書店', '幻冬舎', '朝日新聞出版')


**正規表現による検索**

決まった文字列ではなく、ある規則性を持った文字列を検索したい場合には「正規表現」による検索を用います。ここでは、コミックセットのように複数の書籍が一括して販売されているケースを検索し、「○冊セット」の○にあたる数字を取り出すことを考えます。ここではpythonライブラリ're'のsearchメソッドを一件につき2回用いています。１回目の適用で'○冊セット'の文字列をとりだし、２回目の適用でその中の○にあたる数字の部分を取り出して、int()で数値化しています。

In [86]:
def func_bulk(s):
    """タイトル文字列sの中の冊数を表すパターンを検索して冊数の数値を返す"""
    nitem = 0
    blkstr1 = re.search('[0-9]+巻セット',s)
    if not blkstr1 is None:
        nitem = int(re.search('[0-9]+',blkstr1.group()).group())
        return nitem
    blkstr2 = re.search('[0-9]+冊セット',s)
    if not blkstr2 is None:
        nitem = int(re.search('[0-9]+',blkstr2.group()).group())
        return nitem
    blkstr3 = re.search('全[0-9]+巻',s)
    if not blkstr3 is None:
        nitem = int(re.search('[0-9]+',blkstr3.group()).group())
        return nitem
    blkstr4 = re.search('全[0-9]+冊',s)
    if not blkstr4 is None:
        nitem = int(re.search('[0-9]+',blkstr4.group()).group())
        return nitem
    return nitem

In [87]:
# セット書籍の冊数を取り出す
df1 = df1.assign(n_bulk = df1['title'].apply(func_bulk))
df81 = df1[df1['n_bulk'] > 0]
df81.head(2)

,title,subTitle,seriesName,contents,author,publisherName,size,itemCaption,salesDate,itemPrice,reviewCount,reviewAverage,n_bulk
19,アシガール 1-14巻セット,,マーガレットコミックス,,森本梢子,集英社,,,2020年06月下旬,6776,0,0.0,14
36,100年ドラえもん 50周年メモリアルエディション 『ドラえもん』全45巻・豪華愛蔵版セット,,,,藤子・F・不二雄,小学館,,,2020年12月01日,77000,0,0.0,45


**データフレーム内の複数の列の演算により新たな列を作成する。**

これまでにもデータフレーム内のある一つの列から、ある演算によって新しい列を作り出す例が出てきました。演算が簡単なものならばその列を作り出す文の中に式を埋め込めばいいのですが、式が複雑な場合は別途関数を定義して、元の列に関してapplyメソッドを使う、という形でした。それでは、データフレーム内の複数の列を使って新しい列を作り出す場合はどうするのでしょうか。その場合は、データフレームそのものについてappyメソッドを適用します。

In [88]:
# データフレーム内の複数の列の演算により新しい列を作成する
def func_review_pi(df):
  return np.log(df.loc['reviewCount']+1) + df.loc['reviewAverage']
df51 = df1.assign(review_pi = df1.apply(func_review_pi, axis=1))
df51.head(2)

,title,subTitle,seriesName,contents,author,publisherName,size,itemCaption,salesDate,itemPrice,reviewCount,reviewAverage,n_bulk,review_pi
0,ダンススクエア vol.39,,,,日之出出版,マガジンハウス,ムックその他,,2020年06月27日,980,0,0.00,0,0.000000
1,鬼滅の刃 23巻 フィギュア付き同梱版,,ジャンプコミックス,,吾峠 呼世晴,集英社,コミック,,2020年12月04日,5720,23,4.41,0,7.588054


**新しい列を作成するための関数に他の引数がある場合**

これまでの例では、データフレーム内の新たな列を作るための関数には「元の列」や「元のデータフレーム」以外の引数はありませんでした。今度は、その関数が別途他の引数を必要とする場合の記述を見てみます。
ここでの例は、書籍のタイトルにある指定したキーワードが含まれていたら、あるジャンルの書籍であると判定するための関数の作成です。一つのジャンルについてキーワードは複数あるものとし、キーワードは2次元のリストで与えられるとします。

In [89]:
# ジャンル別のキーワードのリスト
# 0:経済, 1:歴史, 2:科学, 3:医学, 4:旅行, 5:辞典, 6:人生, 7:食事, 8:情報, 9:入試, 10:漫画
kws = [['経済','財務','お金','家計','会計','金融'],
       ['史','古代'],
       ['科学','サイエンス','宇宙'],
       ['医','癌','内科','外科','歯科','手術','疾患'],
       ['旅','紀行'],
       ['辞典','広辞苑'],
       ['人生','生き方','生きかた','ライフ'],
       ['料理','食事','グルメ','レシピ'],
       ['Excel','Word','Python','iPhone','iPad','プログラミング','アプリ','HTML','ソフトウェア','情報技術','Adobe'],
       ['入試','中学受験','高校受験','大学受験'],
       ['鬼滅の刃','リベンジャーズ','るろうに剣心','スライムだった件','青の祓魔師','呪術廻戦','進撃の巨人']]

In [90]:
def func_genre(s, ww, k):
  """文字列sに2次元リストwwの第kジャンルのキーワードが
  含まれていれば1, 含まれていなければ0を返す"""
  q = 0
  for j in range(len(ww[k])):
    if ww[k][j] in s:
      q = 1
  return q
    

演算の元になる列以外の引数がある場合、その引数はapply()メソッドの中で引数名=値の形で指定します。

In [91]:
# 各キーワードを含むかどうかのダミー変数を作成する
df1= df1.assign(G経済=df1['title'].apply(func_genre, ww=kws, k=0))
df1= df1.assign(G歴史=df1['title'].apply(func_genre, ww=kws, k=1))
df1= df1.assign(G科学=df1['title'].apply(func_genre, ww=kws, k=2))
df1= df1.assign(G医学=df1['title'].apply(func_genre, ww=kws, k=3))
df1= df1.assign(G旅行=df1['title'].apply(func_genre, ww=kws, k=4))
df1= df1.assign(G辞典=df1['title'].apply(func_genre, ww=kws, k=5))
df1= df1.assign(G人生=df1['title'].apply(func_genre, ww=kws, k=6))
df1= df1.assign(G食事=df1['title'].apply(func_genre, ww=kws, k=7))
df1= df1.assign(G情報=df1['title'].apply(func_genre, ww=kws, k=8))
df1= df1.assign(G入試=df1['title'].apply(func_genre, ww=kws, k=9))
df1= df1.assign(G漫画=df1['title'].apply(func_genre, ww=kws, k=10))

実際に各ジャンルに該当する書籍が何件あったかを見るには、数値変数に絞り込んで合計を計算します。

In [92]:
# 文字列変数を除外する
df41 = df1.select_dtypes(exclude='object')
# 数値変数の合計をみる
df42 = df41.sum()
df42

itemPrice        5708324.00
reviewCount        16681.00
reviewAverage       4738.91
n_bulk               305.00
G経済                   52.00
G歴史                   46.00
G科学                   24.00
G医学                   62.00
G旅行                   20.00
G辞典                   29.00
G人生                   44.00
G食事                   45.00
G情報                   19.00
G入試                   30.00
G漫画                   41.00
dtype: float64

In [93]:
# アイテム価格の回帰分析
import statsmodels.formula.api as smf
expr = 'itemPrice ~ reviewCount+reviewAverage+n_bulk+G経済+G歴史+G科学+G医学+G旅行+G辞典+G人生+G食事+G情報+G入試+G漫画'
results = smf.ols(expr, data=df41).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              itemPrice   R-squared:                       0.200
Model:                            OLS   Adj. R-squared:                  0.196
Method:                 Least Squares   F-statistic:                     53.23
Date:                Fri, 23 Jul 2021   Prob (F-statistic):          3.54e-133
Time:                        00:25:30   Log-Likelihood:                -28364.
No. Observations:                3000   AIC:                         5.676e+04
Df Residuals:                    2985   BIC:                         5.685e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept      1775.2010     73.002     24.317

In [94]:
# レビュー得点の回帰分析
import statsmodels.formula.api as smf
expr = 'reviewAverage ~ G経済+G歴史+G科学+G医学+G旅行+G辞典+G人生+G食事+G情報+G入試+G漫画'
results = smf.ols(expr, data=df41).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:          reviewAverage   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     9.117
Date:                Fri, 23 Jul 2021   Prob (F-statistic):           3.11e-16
Time:                        00:25:30   Log-Likelihood:                -6531.6
No. Observations:                3000   AIC:                         1.309e+04
Df Residuals:                    2988   BIC:                         1.316e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.5380      0.042     36.817      0.0